<a href="https://colab.research.google.com/github/Me-lab-c/AICTE_internship_garbage_classification/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np  # Importing NumPy for numerical operations and array manipulations
import matplotlib.pyplot as plt  # Importing Matplotlib for plotting graphs and visualizations
import seaborn as sns  # Importing Seaborn for statistical data visualization, built on top of Matplotlib
import tensorflow as tf  # Importing TensorFlow for building and training machine learning models
from tensorflow import keras  # Importing Keras, a high-level API for TensorFlow, to simplify model building
from tensorflow.keras import Layer  # Importing Layer class for creating custom layers in Keras
from tensorflow.keras.models import Sequential  # Importing Sequential model for building neural networks layer-by-layer
from tensorflow.keras.layers import Rescaling , GlobalAveragePooling2D
from tensorflow.keras import layers, optimizers, callbacks  # Importing various modules for layers, optimizers, and callbacks in Keras
from sklearn.utils.class_weight import compute_class_weight  # Importing function to compute class weights for imbalanced datasets
from tensorflow.keras.applications import EfficientNetV2B2  # Importing EfficientNetV2S model for transfer learning
from sklearn.metrics import confusion_matrix, classification_report  # Importing functions to evaluate model performance
import gradio as gr  # Importing Gradio for creating interactive web interfaces for machine learning models
from tensorflow.keras.utils import image_dataset_from_directory  # Importing function to load image data from a directory
from tensorflow.keras.layers import Dense  # Importing Dense layer for fully connected layers in neural networks

In [ ]:
#code for directly asseces dataset fron kaagle

!pip install opendatasets
import opendatasets as od
import os # Import the os module

In [ ]:
# Replace 'your_kaggle_dataset_url' with the URL of the Kaggle dataset
dataset_url = 'https://www.kaggle.com/datasets/mostafaabla/garbage-classification'
od.download(dataset_url)

Skipping, found downloaded files in "./garbage-classification" (use force=True to force download)


In [ ]:
# Define the path to the downloaded dataset directory
dataset_dir = '/content/garbage-classification'

In [ ]:
# Define image size and batch size
img_height = 260
img_width = 260
batch_size = 32

In [ ]:
# Load the training, validation, and test datasets
train_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 15515 files belonging to 1 classes.
Using 12412 files for training.
Found 15515 files belonging to 1 classes.
Using 3103 files for validation.


In [ ]:
# Get the number of classes
num_classes = len(train_ds.class_names)
print(f"Number of classes: {num_classes}")

# Build the model
base_model = EfficientNetV2B2(input_shape=(img_height, img_width, 3),
                              include_top=False,
                              weights='imagenet')

Number of classes: 1


In [ ]:
# Freeze the base model
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax') # Use softmax for multi-class classification
])

# Compile the model (example using Adam optimizer)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # Use this if your labels are integers
              metrics=['accuracy'])